# OCR for printed Alphanumeric characters

Environment - Anaconda Python 3.6 - Name: `Keras`
<br> Install the following after activating your Conda environment
* `tensorflow`
* `keras`
* `ipykernel`

<br> Set your Conda environment as a kernel for the Jupyter Notebook <br>
`python -m ipykernel install --user --name=Keras`
<br> Navigate to folder containing notebook & data and start your Jupyter server <br>
`jupyter notebook`
<br> Start the notebook, and choose `Kernel` -> `Restart and Run All`
<br> Directory structure for data - `~/English/Fnt/SampleXXX/imgXXX-YYYYY.png`

In [ ]:
import numpy as np
import cv2
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import random
import keras.backend as K
K.set_image_data_format('channels_last')
import keras


In [ ]:
X_train=[]
X_test=[]
Y_train=[]
Y_test=[]

for i in range(1,37):
    s="%03d"%i
    directory="./English/Fnt/Sample"+s
    for j in range(1,1017):
        s2="%05d"%j
        img_name=directory+"/img"+s+"-"+s2+".png"
        img=cv2.imread(img_name)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_CUBIC)
        if j<=762:
            X_train.append(img)
        else:
            X_test.append(img)

In [ ]:
X_train=np.asarray(X_train)
X_test=np.asarray(X_test) 
for i in range(0,36):
    for k in range(1016):
        temp=np.zeros(36)
        temp[i]=1
        if k<762:
            Y_train.append(temp)
        else:
            Y_test.append(temp)
Y_train=np.asarray(Y_train)
Y_test=np.asarray(Y_test)

In [ ]:
img_rows, img_cols = 32, 32

In [ ]:
input_shape = (img_rows, img_cols, 1)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)

In [ ]:
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
num_classes = 36

In [ ]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
epochs = 10
batch_size = 128

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])